# Chapter 8: Calculation of Molecular Properties

## 8.7. Molecular Dynamics Simulation

In the following section, we will perform molecular dynamics simulation for organic molecules:

### 8.7.1. *Ab initio* calculation

In [ ]:
# Import modules
import numpy as np
import psi4
from rdkit import Chem
from rdkit.Chem import AllChem
from utils import View3DModel
import py3Dmol
from tqdm import tqdm

In [ ]:
# Create a molecule of acetone
mol = Chem.MolFromSmiles('CC(=O)C') # acetone

# Prepare the molecule
mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol, useRandomCoords=True)
AllChem.UFFOptimizeMolecule(mol, maxIters=200)

In [ ]:
# View 3D model with py3Dmol
View3DModel(mol)

In [ ]:
# Write the geometry to XYZ string
xyz_string = Chem.MolToXYZBlock(mol)

# Get the psi4 geometry
geometry = psi4.geometry(xyz_string)

# Convert initial coordinates to numpy array
natoms = geometry.natom()
coords = np.array([[geometry.x(i), geometry.y(i), geometry.z(i)] for i in range(natoms)])

# Initial velocities = 0
velocities = np.zeros_like(coords)

In [ ]:
# Simulation parameters
dt = 0.5  # time step in fs (arbitrary units for demo)
num_steps = 100

# Store PDB trajectory
pdb_traj = ""

In [ ]:
# Set calculation options
psi4.set_options({'BASIS': '6-31g'})

# Set the number of threads and memory limit
psi4.set_num_threads(16)
psi4.set_memory(16*1024*1024*1024) # 16 GB

In [ ]:
# Keep initial reference positions
ref_coords = coords.copy()

# Damping factor for velocities (to avoid runaway)
damping = 0.9

# Harmonic restraint constant (a.u.)
k_restraint = 0.05

In [ ]:
for step in tqdm(range(1, num_steps+1)):
    # Update Psi4 molecule geometry
    geometry = psi4.geometry(
        f"0 1\n" +
        "\n".join([f"{geometry.symbol(i)} {coords[i,0]} {coords[i,1]} {coords[i,2]}" for i in range(natoms)])
    )
    
    # Compute forces (negative gradient from QM)
    grad = psi4.gradient('SCF', molecule=geometry).to_array()
    forces = -grad  # Hartree/Bohr
    
    # Add harmonic restraint force: -k * (x - x0)
    restraint_force = -k_restraint * (coords - ref_coords)
    forces += restraint_force
    
    # Velocity Verlet with damping
    velocities = damping * (velocities + 0.5 * forces * dt)
    coords += velocities * dt
    
    # Write step to PDB string
    conformer = mol.GetConformer()
    pdb_traj += f"MODEL     {step}\n"
    for i in range(natoms):
        pdb_traj += f"ATOM  {i+1:5d} {geometry.symbol(i):>2}   MOL     1    {coords[i,0]:8.3f}{coords[i,1]:8.3f}{coords[i,2]:8.3f}  1.00  0.00\n"
        conect_lines = ""
        for bond in mol.GetBonds():
            a1 = bond.GetBeginAtomIdx() + 1  # PDB atoms are 1-indexed
            a2 = bond.GetEndAtomIdx() + 1
            conect_lines += f"CONECT{a1:5d}{a2:5d}\n"
    pdb_traj += conect_lines
    pdb_traj += "ENDMDL\n"

In [ ]:
# Show the animation
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(pdb_traj, "trajectory", {'keepH': True})
view.setBackgroundColor('white')
view.setStyle({'stick': {'scale': 0.3}, 'sphere': {'scale': 0.3}})
view.zoomTo()
view.animate({'loop': "forward", 'interval': 100}) # Adjust the speed as needed (set 'interval' to a new value in millisecond)
view.show()

### 8.7.2. DFT Calculation

In [ ]:
# Import modules
import numpy as np
import psi4
from rdkit import Chem
from rdkit.Chem import AllChem
from utils import View3DModel
import py3Dmol
from tqdm import tqdm

In [ ]:
# Create a molecule of acetone
mol = Chem.MolFromSmiles('CC(=O)C') # acetone

# Prepare the molecule
mol = Chem.AddHs(mol)
AllChem.EmbedMolecule(mol, useRandomCoords=True)
AllChem.UFFOptimizeMolecule(mol, maxIters=200)

In [ ]:
# View 3D model with py3Dmol
View3DModel(mol)

In [ ]:
# Write the geometry to XYZ string
xyz_string = Chem.MolToXYZBlock(mol)

# Get the psi4 geometry
geometry = psi4.geometry(xyz_string)

# Convert initial coordinates to numpy array
natoms = geometry.natom()
coords = np.array([[geometry.x(i), geometry.y(i), geometry.z(i)] for i in range(natoms)])

# Initial velocities = 0
velocities = np.zeros_like(coords)

In [ ]:
# Simulation parameters
dt = 0.5  # time step in fs (arbitrary units for demo)
num_steps = 100

# Store PDB trajectory
pdb_traj = ""

In [ ]:
# Set calculation options
psi4.set_options({'BASIS': '6-31g', 'SCF_TYPE': 'DF'})

# Set the number of threads and memory limit
psi4.set_num_threads(16)
psi4.set_memory(16*1024*1024*1024) # 16 GB

In [ ]:
# Keep initial reference positions
ref_coords = coords.copy()

# Damping factor for velocities (to avoid runaway)
damping = 0.9

# Harmonic restraint constant (a.u.)
k_restraint = 0.05

In [ ]:
for step in tqdm(range(1, num_steps+1)):
    # Update Psi4 molecule geometry
    geometry = psi4.geometry(
        f"0 1\n" +
        "\n".join([f"{geometry.symbol(i)} {coords[i,0]} {coords[i,1]} {coords[i,2]}" for i in range(natoms)])
    )
    
    # Compute forces (negative gradient from QM)
    grad = psi4.gradient('B3LYP', molecule=geometry).to_array()
    forces = -grad  # Hartree/Bohr
    
    # Add harmonic restraint force: -k * (x - x0)
    restraint_force = -k_restraint * (coords - ref_coords)
    forces += restraint_force
    
    # Velocity Verlet with damping
    velocities = damping * (velocities + 0.5 * forces * dt)
    coords += velocities * dt
    
    # Write step to PDB string
    conformer = mol.GetConformer()
    pdb_traj += f"MODEL     {step}\n"
    for i in range(natoms):
        pdb_traj += f"ATOM  {i+1:5d} {geometry.symbol(i):>2}   MOL     1    {coords[i,0]:8.3f}{coords[i,1]:8.3f}{coords[i,2]:8.3f}  1.00  0.00\n"
        conect_lines = ""
        for bond in mol.GetBonds():
            a1 = bond.GetBeginAtomIdx() + 1  # PDB atoms are 1-indexed
            a2 = bond.GetEndAtomIdx() + 1
            conect_lines += f"CONECT{a1:5d}{a2:5d}\n"
    pdb_traj += conect_lines
    pdb_traj += "ENDMDL\n"

In [ ]:
# Show the animation
view = py3Dmol.view(width=400, height=300)
view.addModelsAsFrames(pdb_traj, "trajectory", {'keepH': True})
view.setBackgroundColor('white')
view.setStyle({'stick': {'scale': 0.3}, 'sphere': {'scale': 0.3}})
view.zoomTo()
view.animate({'loop': "forward", 'interval': 100}) # Adjust the speed as needed (set 'interval' to a new value in millisecond)
view.show()